**Support code for Exploratory Report** <br>
https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca

In [1]:
#!pip install web3

In [2]:
import json
from datetime import datetime
from decimal import Decimal
import pandas as pd
from web3 import Web3
from web3.exceptions import (InvalidEventABI, LogTopicError, MismatchedABI)
from web3._utils.events import get_event_data
from eth_utils import (encode_hex, event_abi_to_log_topic)


# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
print ('Ethereum connected:', w3_eth.isConnected())


Ethereum connected: True


In [3]:
abi = json.loads("""[
                     {"name":"decimals",
                      "inputs":[],"outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view",
                      "type":"function"},
                     {"name":"AnswerUpdated",
                      "anonymous":false,
                      "inputs":[{"indexed":true,"internalType":"int256","name":"current","type":"int256"},{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"updatedAt","type":"uint256"}],
                      "type":"event"}   
                    ]
                """)

pricefeed_contracts = {
     'eth-usd' : [w3_eth, Web3.toChecksumAddress('0x37bC7498f4FF12C19678ee8fE19d713b87F6a9e6'), 0.005, 'usd', 3600]
    
    ,'btc-eth' : [w3_eth, Web3.toChecksumAddress('0x81076d6ff2620ea9dd7ba9c1015f0d09a3a732e6'), 0.02, 'eth', 86400]
    ,'btc-usd' : [w3_eth, Web3.toChecksumAddress('0xAe74faA92cB67A95ebCAB07358bC222e33A34dA7'), 0.005, 'usd', 3600]
    
    ,'usdc-eth': [w3_eth, Web3.toChecksumAddress('0xe5bbbdb2bb953371841318e1edfbf727447cef2e'), 0.01, 'eth', 86400]
    ,'usdc-usd': [w3_eth, Web3.toChecksumAddress('0x789190466E21a8b78b8027866CBBDc151542A26C'), 0.0025, 'usd', 86400]
    
    ,'dai-eth' : [w3_eth, Web3.toChecksumAddress('0x158228e08c52f3e2211ccbc8ec275fa93f6033fc'), 0.01, 'eth', 86400]
    ,'dai-usd' : [w3_eth, Web3.toChecksumAddress('0xDEc0a100eaD1fAa37407f0Edc76033426CF90b82'), 0.0025, 'usd', 3600]
    
    ,'1inch-eth': [w3_eth, Web3.toChecksumAddress('0xb2f68c82479928669b0487d1daed6ef47b63411e'), 0.02, 'eth', 86400]
    ,'1inch-usd': [w3_eth, Web3.toChecksumAddress('0xd2bdD1E01fd2F8d7d42b209c111c7b32158b5a42'), 0.01, 'usd', 86400]
    
    ,'aave-eth': [w3_eth, Web3.toChecksumAddress('0xdf0da6b3d19e4427852f2112d0a963d8a158e9c7'), 0.02, 'eth', 86400]
    ,'aave-usd': [w3_eth, Web3.toChecksumAddress('0xe3f0dEdE4B499c07e12475087AB1A084b5F93bc0'), 0.01, 'usd', 3600]
    
    ,'comp-eth': [w3_eth, Web3.toChecksumAddress('0x9d6acd34d481512586844fd65328bd358d306752'), 0.02, 'eth', 86400]     
    ,'comp-usd': [w3_eth, Web3.toChecksumAddress('0x6eaC850f531d0588c0114f1E93F843B78669E6d2'), 0.01, 'usd', 3600]     
    
    ,'dpi-eth': [w3_eth, Web3.toChecksumAddress('0x36e4f71440edf512eb410231e75b9281d4fcfc4c'), 0.02, 'eth', 86400]     
    ,'dpi-usd': [w3_eth, Web3.toChecksumAddress('0x68f1b8317c19ff02fb68a8476c1d3f9fc5139c0a'), 0.01, 'usd', 3600]     

    ,'fei-eth': [w3_eth, Web3.toChecksumAddress('0x4be991b4d560bba8308110ed1e0d7f8da60acf6a'), 0.02, 'eth', 86400] 
    ,'fei-usd': [w3_eth, Web3.toChecksumAddress('0x1D244648d5a63618751d006886268Ae3550d0Dfd'), 0.01, 'usd', 3600] 
    
    ,'link-eth': [w3_eth, Web3.toChecksumAddress('0xbba12740de905707251525477bad74985dec46d2'), 0.01, 'eth', 21600] 
    ,'link-usd': [w3_eth, Web3.toChecksumAddress('0xDfd03BfC3465107Ce570a0397b247F546a42D0fA'), 0.01, 'usd', 3600] 

    ,'snx-eth': [w3_eth, Web3.toChecksumAddress('0xbafe3cb0e563e914806a99d547bdbf2cfcf5fdf6'), 0.02, 'eth', 86400] 
    ,'snx-usd': [w3_eth, Web3.toChecksumAddress('0x06ce8Be8729B6bA18dD3416E3C223a5d4DB5e755'), 0.01, 'usd', 86400] 

    ,'uni-eth': [w3_eth, Web3.toChecksumAddress('0xc1d1d0da0fcf78157ea25d0e64e3be679813a1f7'), 0.02, 'eth', 86400] 
    ,'uni-usd': [w3_eth, Web3.toChecksumAddress('0x68577f915131087199Fe48913d8b416b3984fd38'), 0.01, 'usd', 3600] 

    ,'yfi-eth': [w3_eth, Web3.toChecksumAddress('0xaa5aa80e416f9d32ffe6c390e24410d02d203f70'), 0.01, 'eth', 86400]
    ,'yfi-usd': [w3_eth, Web3.toChecksumAddress('0x8a4D74003870064d41D4f84940550911FBfCcF04'), 0.01, 'usd', 3600]
    
    ,'luna-eth': [w3_eth, Web3.toChecksumAddress('0x1a8ac67a1b64f7fd71bb91c21581f036abe6aec2'), 0.02, 'eth', 86400]
    
    ,'ftm-eth': [w3_eth, Web3.toChecksumAddress('0xbdb80d19dea36eb7f63bdfd2bdd4033b2b7e8e4d'), 0.03, 'eth', 86400]
     
    ,'sushi-eth': [w3_eth, Web3.toChecksumAddress('0xd01bbb3afed2cb5ca92ca3834d441dc737f0da70'), 0.02, 'eth', 86400]
    ,'sushi-usd': [w3_eth, Web3.toChecksumAddress('0x7213536a36094cD8a768a5E45203Ec286Cba2d74'), 0.01, 'usd', 3600]
    
    ,'crv-eth': [w3_eth, Web3.toChecksumAddress('0x7f67ca2ce5299a67acd83d52a064c5b8e41ddb80'), 0.02, 'eth', 86400]
    ,'crv-usd': [w3_eth, Web3.toChecksumAddress('0xb4c4a493AB6356497713A78FFA6c60FB53517c63'), 0.01, 'usd', 86400]
    
    ,'cvx-usd': [w3_eth, Web3.toChecksumAddress('0x8d73ac44bf11cadcdc050bb2bccae8c519555f1a'), 0.02, 'usd', 86400] 
    
    ,'ldo-eth': [w3_eth, Web3.toChecksumAddress('0x7898AcCC83587C3C55116c5230C17a6Cd9C71bad'), 0.02, 'eth', 86400] 
    
    ,'steth-eth': [w3_eth, Web3.toChecksumAddress('0x716BB759A5f6faCdfF91F0AfB613133d510e1573'), 0.02, 'eth', 86400] 
    ,'steth-usd': [w3_eth, Web3.toChecksumAddress('0xdA31bc2B08F22AE24aeD5F6EB1E71E96867BA196'), 0.01, 'usd', 3600] 
    
    ,'frax-eth': [w3_eth, Web3.toChecksumAddress('0x56f98706C14DF5C290b02Cec491bB4c20834Bb51'), 0.02, 'eth', 86400] 
    ,'frax-usd': [w3_eth, Web3.toChecksumAddress('0x61eB091ea16A32ea5B880d0b3D09d518c340D750'), 0.01, 'usd', 3600] 

    ,'lusd-usd': [w3_eth, Web3.toChecksumAddress('0x27b97a63091d185cE056e1747624b9B92BAAD056'), 0.01, 'usd', 3600] 

    ,'susd-eth': [w3_eth, Web3.toChecksumAddress('0x45bb69B89D60878d1e42522342fFCa9F2077dD84'), 0.01, 'eth', 86400] 
    ,'susd-usd': [w3_eth, Web3.toChecksumAddress('0x1187272A0E3A603eC4734CeC73a0880055eCC593'), 0.005, 'usd', 86400] 

    ,'gusd-eth': [w3_eth, Web3.toChecksumAddress('0x9c2C487DAd6C8e5bb49dC6908a29D95a234FaAd8'), 0.02, 'eth', 86400] 
    ,'gusd-usd': [w3_eth, Web3.toChecksumAddress('0x6a805f2580b8D75d40331c26C074c2c42961E7F2'), 0.005, 'usd', 86400] 

    ,'usdt-eth': [w3_eth, Web3.toChecksumAddress('0x7De0d6fce0C128395C488cb4Df667cdbfb35d7DE'), 0.01, 'eth', 86400] 
    ,'usdt-usd': [w3_eth, Web3.toChecksumAddress('0xa964273552C1dBa201f5f000215F5BD5576e8f93'), 0.0025, 'usd', 86400] 

    ,'fxs-usd': [w3_eth, Web3.toChecksumAddress('0x9d78092775DFE715dFe1b0d71aC1a4d6e3652559'), 0.02, 'usd', 86400] 
    
    ,'spell-usd': [w3_eth, Web3.toChecksumAddress('0x8640b23468815902e011948F3aB173E1E83f9879'), 0.01, 'usd', 86400] 
    
    }

config = {'Ethereum':pricefeed_contracts, 
         }

In [4]:
def get_event_abi(abi, abi_name, abi_type = 'event'):
    l = [x for x in abi if x['type'] == abi_type and x['name']==abi_name]
    return l[0]
def get_logs (w3, contract_address, topics, events, from_block=0, to_block=None):

    if not to_block:
        to_block = w3.eth.get_block('latest').number
    
    try:
        logs = w3.eth.get_logs({"address": contract_address
                               ,"topics":topics 
                               ,"fromBlock": from_block 
                               ,"toBlock": to_block
                               })
    except ValueError:
        logs = None
        from_block = 12*10**6
        batch_size =  3*10**5
        while from_block < to_block:
            print(datetime.utcnow(), f'from_block {from_block} to_block {from_block + batch_size}')
            batch_logs = w3.eth.get_logs({"address": contract_address
                                   ,"topics":topics 
                                   ,"fromBlock": from_block 
                                   ,"toBlock": from_block + batch_size # narrow down the range to avoid too many results error 
                                   })   
            if not logs:
                logs = batch_logs
            else:
                logs = logs + batch_logs
            if len(logs) > 0:    
                print(datetime.utcnow(), f'Total logs count {len(logs)}')  
            
            from_block = from_block + batch_size + 1
            
    all_events = []
    for l in logs:
        try:
            evt_topic0 = l['topics'][0].hex()            
            evt_abi = [x for x in events if encode_hex(event_abi_to_log_topic(x)) == evt_topic0][0]
            evt = get_event_data(w3.codec, evt_abi, l)
        except MismatchedABI: #if for some reason there are other events 
            pass
        all_events.append(evt)
    df = pd.DataFrame(all_events)
    return df

In [5]:
from google.oauth2 import service_account
import pandas_gbq

gcp_project_id = 'gearbox-336415'
from_block = 0
try:
    df_blocknum = pandas_gbq.read_gbq('select max(blockNumber) from gearbox.oracle_price_history', 
                             project_id=gcp_project_id,
                             progress_bar_type = None,)
    from_block = int(df_blocknum.iloc[0,0]) + 1
except pandas_gbq.exceptions.GenericGBQException:
    print('The table does not exist?')
    from_block = 0

#from_block = 0
print(datetime.utcnow(),'from_block=', from_block)

df = pd.DataFrame()

event_AnswerUpdated = get_event_abi(abi, 'AnswerUpdated')
topic_AnswerUpdated  = encode_hex(event_abi_to_log_topic(event_AnswerUpdated))

for chain in config.keys():
    chain_contracts = config[chain]
    for ticker in chain_contracts.keys():
        #print(f'{datetime.utcnow()} : {ticker.upper()}')
        w3 = chain_contracts[ticker][0]
        address = chain_contracts[ticker][1]
        threshold = chain_contracts[ticker][2]
        base = chain_contracts[ticker][3]
        heartbeat = chain_contracts[ticker][4]
        
        decimals = w3.eth.contract(address=address, abi=abi).functions.decimals().call()

        df_ticker = get_logs(w3, address, [topic_AnswerUpdated], [event_AnswerUpdated], from_block)
        df_ticker[['chain', 'ticker', 'threshold', 'base', 'heartbeat', 'decimals', 'type']] = [chain , ticker, threshold, base, heartbeat, decimals, 'direct']
        
        df_ticker['tx_hash']       = df_ticker['transactionHash'].apply(lambda x: x.hex())        
        df_ticker['updated_at']    = df_ticker['args'].apply(lambda x: pd.to_datetime(x['updatedAt'], unit='s'))
        df_ticker['price']         = df_ticker['args'].apply(lambda x: x['current'])
        df_ticker['price_decimal'] = df_ticker['price']/10**(decimals)
        df_ticker['deviation']     = df_ticker['price']/df_ticker['price'].shift(1) - 1
        
        df = df.append(df_ticker[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'decimals', 'blockNumber','tx_hash']], ignore_index=True)
        print(f'''{datetime.utcnow()} : {ticker.upper()}: observation count = {len(df_ticker)}, price range: {df_ticker['price_decimal'].min()}-{df_ticker['price_decimal'].max()}''')
        
        if ticker == 'eth-usd':
            df_eth_usd = df_ticker
        elif base == 'usd': # usd2eth cross rate
            ticker = ticker.replace('-usd','-eth')
            if ticker not in chain_contracts.keys():
                #print(f'{datetime.utcnow()}: cross rate {ticker.upper()}')
                df_cross = pd.DataFrame()
                for k, t in df_ticker.iterrows(): 
                    e = df_eth_usd[df_eth_usd['updated_at'] <= t['updated_at']]
                    if len(e) > 0:
                        i = (t['updated_at'] - e['updated_at']).idxmin() 
                        e_price_decimal, e_decimals = e.loc[i][['price_decimal','decimals']]
                        t.ticker = ticker                       
                        t.price_decimal = t.price_decimal/e_price_decimal
                        t.decimals = 18
                        t.base = 'eth'                         
                        t.type = 'cross'
                        t.price = int(t.price_decimal*10**(t.decimals))
                        df_cross = df_cross.append(t)
                df_cross['deviation']     = df_cross['price']/df_cross['price'].shift(1) - 1
                df = df.append(df_cross[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'decimals', 'blockNumber','tx_hash']], ignore_index=True)
                print(f'''{datetime.utcnow()}: cross rate {ticker.upper()}: observation count = {len(df_cross)}, price range: {df_cross['price_decimal'].min()}-{df_cross['price_decimal'].max()}''')
        elif base == 'eth': #eth2usd cross rate
            ticker = ticker.replace('-eth','-usd')
            if ticker not in chain_contracts.keys():
                #print(f'{datetime.utcnow()}: cross rate {ticker.upper()}')
                df_cross = pd.DataFrame()
                for k, t in df_ticker.iterrows(): 
                    e = df_eth_usd[df_eth_usd['updated_at'] <= t['updated_at']]
                    if len(e) > 0:
                        i = (t['updated_at'] - e['updated_at']).idxmin() 
                        e_price_decimal, e_decimals = e.loc[i][['price_decimal','decimals']]
                        t.ticker = ticker
                        t.decimals = e_decimals
                        t.base = 'usd'                        
                        t.type = 'cross'
                        t.price_decimal = t.price_decimal*e_price_decimal
                        t.price = int(t.price_decimal*10**(t.decimals))
                        df_cross = df_cross.append(t)
                df_cross['deviation']     = df_cross['price']/df_cross['price'].shift(1) - 1
                df = df.append(df_cross[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'blockNumber','tx_hash']], ignore_index=True)
                print(f'''{datetime.utcnow()}: cross rate {ticker.upper()}: observation count = {len(df_cross)}, price range: {df_cross['price_decimal'].min()}-{df_cross['price_decimal'].max()}''')
                
df = df.sort_values(by = 'updated_at').reset_index(drop=True)
pd.set_option("precision", 18)

2022-05-11 21:54:36.004863 from_block= 14753557
2022-05-11 21:54:37.012088 : ETH-USD: observation count = 123, price range: 2029.3320712-2444.82
2022-05-11 21:54:37.699402 : BTC-ETH: observation count = 2, price range: 13.440363369999998-13.7094127
2022-05-11 21:54:38.512272 : BTC-USD: observation count = 107, price range: 28117.64-31959.98
2022-05-11 21:54:39.239504 : USDC-ETH: observation count = 43, price range: 0.00041293755-0.000494726354120797
2022-05-11 21:54:39.941802 : USDC-USD: observation count = 14, price range: 1.00021142-1.00895921
2022-05-11 21:54:40.659320 : DAI-ETH: observation count = 38, price range: 0.00040912-0.000493131844956322
2022-05-11 21:54:41.368676 : DAI-USD: observation count = 29, price range: 0.99846727-1.00297471
2022-05-11 21:54:42.063090 : 1INCH-ETH: observation count = 17, price range: 0.00041497-0.000491885514769515
2022-05-11 21:54:42.877215 : 1INCH-USD: observation count = 42, price range: 0.94489453-1.04310292
2022-05-11 21:54:43.580508 : AAVE-ET

In [6]:
display(df.head())
display(df.tail())

,ticker,updated_at,chain,price,price_decimal,type,deviation,threshold,base,heartbeat,decimals,blockNumber,tx_hash
0,snx-eth,2022-05-11 07:33:18,Ethereum,1242629200000000.0,0.001242629200000000,direct,NaN,0.020,eth,86400.0,18.0,14753558.0,0xf323e746be2e7e935d437c7f852e99b142c978fafbb6...
1,btc-usd,2022-05-11 07:33:18,Ethereum,3051932000000.0,30519.319999999999708962,direct,NaN,0.005,usd,3600.0,8.0,14753558.0,0xf256d708422fc0c75affe71e76a114ae0571d86349aa...
2,luna-eth,2022-05-11 07:33:18,Ethereum,2089471320594561.0,0.002089471320594561,direct,NaN,0.020,eth,86400.0,18.0,14753558.0,0xc9072675f524c010dc91fe93c628e088c7ac5c7dfff7...
3,susd-eth,2022-05-11 07:33:18,Ethereum,431876310000000.0,0.000431876310000000,direct,NaN,0.010,eth,86400.0,18.0,14753558.0,0x595f5147fcd0bc8af6be649892fde9f2bf0e2a829220...
4,link-usd,2022-05-11 07:33:36,Ethereum,809351227.0,8.093512269999999731,direct,NaN,0.010,usd,3600.0,8.0,14753560.0,0xd50fc388f69ea576823fc74e009d6afca4983f20b685...


,ticker,updated_at,chain,price,price_decimal,type,deviation,threshold,base,heartbeat,decimals,blockNumber,tx_hash
2614,uni-usd,2022-05-11 21:53:41,Ethereum,481936830.0,4.819368299999999827,direct,0.012472331932773084,0.01,usd,3600.0,8.0,14757342.0,0xafa74c8f534094cb481c753b776f9e6ffdff4be89c69...
2615,aave-usd,2022-05-11 21:54:04,Ethereum,8087128084.0,80.871280839999997170,direct,0.014000536814441134,0.01,usd,3600.0,8.0,14757343.0,0x8d950411caa2c727679781c55a359e8b7d5f69fd3dbc...
2616,usdt-eth,2022-05-11 21:54:15,Ethereum,480776387308034.0,0.000480776387308034,direct,-0.014071154847406842,0.01,eth,86400.0,18.0,14757344.0,0x62fc0bf71ee27f8f2bf4fd7cd58f220461b3435b14e0...
2617,sushi-usd,2022-05-11 21:54:22,Ethereum,128896265.0,1.288962649999999988,direct,0.016070731797316329,0.01,usd,3600.0,8.0,14757345.0,0x15494fde49fe669f0b562f8d6ede9f1ba7784408a4f0...
2618,crv-eth,2022-05-11 21:54:51,Ethereum,708725553535433.0,0.000708725553535433,direct,0.020114235251207147,0.02,eth,86400.0,18.0,14757347.0,0x49f0f656d7c50908b67e9db91536b295cc23e567eef7...


**Write data frame to GCP BigQuery**

In [7]:
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'
if len(df)>0:
    numeric_cols = [x for x in df.columns if x in ['price', 'price_decimal' ,'deviation']]
    df[numeric_cols] = df[numeric_cols].astype('float64')

    pandas_gbq.to_gbq(df, 
                      'gearbox.oracle_price_history',
                      project_id=gcp_project_id,
                      if_exists = ('replace' if from_block==0 else 'append'), 
                      progress_bar = False)
    print(datetime.utcnow(), 'gearbox.oracle_price_history, insert done')
else:
    print('No new events since block', from_block)

2022-05-11 21:55:20.320769 gearbox.oracle_price_history, insert done


# Exploratory report:

https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca